In [1]:
%load_ext autoreload
%autoreload 2
from Crawl_Naver_Shopping import *
import pickle
from lxml import etree
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
category_df = pd.read_excel("수집_대상_카테고리.xlsx")

In [3]:
category_df["cat_levels"] = category_df["category1"].fillna("") + ">" +  category_df["category2"].fillna("") + ">" + category_df["category3"].fillna("") + ">" + category_df["category4"].fillna("")

In [4]:
category_df["cat_levels"]

0                   디지털/가전>노트북>>
1                    디지털/가전>PC>>
2                   디지털/가전>모니터>>
3                디지털/가전>영상가전>TV>
4              디지털/가전>영상가전>프로젝터>
                 ...            
188    디지털/가전>생활가전>건조기/탈수기>신발건조기
189       출산/육아>육아발육용품>바운서/흔들침대>
190             출산/육아>수유용품>보틀워머>
191        생활/건강>주방용품>커피용품>우유거품기
192         출산/육아>위생/건강용품>콧물흡입기>
Name: cat_levels, Length: 193, dtype: object

In [5]:
crawl = Crawl_Naver_Shopping(list_type="가격비교")
crawl_result = crawl.get_product_infos_in_category(cat_ids = category_df["category_id"].to_list(), cat_levels = category_df["cat_levels"].to_list())

0, 디지털/가전>노트북>>


  4%|▍         | 4/100 [00:04<01:43,  1.07s/it]

In [7]:
pkl_list = [x for x in os.listdir("pickle/") if "pkl" in x]

In [8]:
for i, pkl in enumerate(pkl_list):
    with open(f"pickle/{pkl}", "rb") as f:
        if i == 0:
            result = pd.DataFrame(pickle.load(f))
        else:
            temp = pd.DataFrame(pickle.load(f))
            result = pd.concat([result, temp], axis = 0)

In [11]:
crawl_df = pd.DataFrame(crawl_result)

In [12]:
def extract_price_list(x):
    if x is None:
        return None
    else:
        return [mall_info.get("price") for mall_info in x]

In [13]:
f"{10000000:,}"

'10,000,000'

In [14]:
crawl_df["price_list"] = crawl_df["lowMallList"].apply(extract_price_list)

In [15]:

from datetime import datetime
today = datetime.today().strftime("%y.%m.%d")
today

'23.06.22'

In [16]:
def add_price_attr(df):
    if df["lowMallList"] is not None:
        price_list = [int(mall_info.get("price")) for mall_info in df["lowMallList"]]

        new_product_attr = df["productAttr"] + " | " + f"가격 : {min(price_list):,} ~ {max(price_list):,} ({today} 기준)"
        return new_product_attr
    else:
        return df["productAttr"]
crawl_df["productAttr"] = crawl_df.apply(add_price_attr, axis = 1)

In [31]:
crawl_df.shape

(337080, 29)

In [17]:
crawl_df.to_excel("naver_shopping_230622.xlsx")

In [20]:
crawl_df.loc[crawl_df["productName"].str.contains("27LX5QKNA")]

,rank,id,scoreInfo,productName,overseaTp,saleTp,category1Id,category2Id,category3Id,category4Id,...,lnchYm,lowMallList,attributeValue,characterValue,crUrl,imageUrl,imageName,productAttr,searchCategoryId,price_list
35,36,40498606618,NaN,27LX5QKNA,0,0,50000003,50000208,50001395,50002114,...,20230601,"[{'nvMid': '40482053756', 'mallSeq': '613264',...",종류_M|TV화면크기_M|해상도_M|스마트_M|부가기능_M|부가기능_M|부가기능_M...,LED|68.58cm(27인치)|Full HD|음성인식|WiFi|블루투스5.0|HD...,https://cr.shopping.naver.com/adcr.nhn?x=7ClJQ...,https://shopping-phinf.pstatic.net/main_404986...,40498606618.20230609130229.jpg,종류 : LED | TV화면크기 : 68.58cm(27인치) | 해상도 : Full...,50001395,"[2000000, 2820000]"


In [ ]:
# 스탠드 제품 코드 추가
stand_ids = ["50003330", "50003331"]
stand_levels = [["가구/인테리어", "조명/스탠드", "스탠드", "단스탠드"], 
 ["가구/인테리어", "조명/스탠드", "스탠드", "장스탠드"]]

unq_category_ids.extend(stand_ids)
unq_category_levels.extend(stand_levels)

In [ ]:
category_list = ["LEDTV", "OLEDTV", "LCDTV", "QLEDTV", "DLP", "LCD", "LCoS/기타", 
"퍼스널뷰어", "블루레이플레이어", "사운드바시스템", "단일형홈시어터", 
"조합형홈시어터", "일체형오디오", "스피커단품", "스피커세트", "블루투스스피커", 
"블루투스이어폰/이어셋", "블루투스헤드폰/헤드셋", "이어폰", "헤드폰", 
"일반세탁기", "드럼세탁기", "미니세탁기", "세탁기+건조기", "의류건조기", 
"신발건조기", "탈수기", "유선청소기", "무선청소기", "스팀청소기", 
"로봇청소기", "물걸레청소기", "연수기", "단스탠드", "장스탠드", "의류관리기", 
"스팀다리미", "보풀제거기", "전동칫솔", "구강세정기", "칫솔살균기", 
"전신건조기", "드라이어", "고데기", "매직기", "에어브러시", "남성용면도기", 
"여성용면도기", "두피케어기기", "제모기", "양문형냉장고", "일반형냉장고", 
"스탠드형", "냉동고", "와인냉장고", "화장품냉장고", "쌀냉장고", "제빙기", 
"일반가스레인지", "일반밥솥", "전자레인지", "전기오븐", "복합형오븐", "인덕션",
"핫플레이트", "하이라이트", "하이브리드", "식기세척기", "식기건조기", "무선포트", 
"유선포트", "냉정수기", "냉온정수기", "에스프레소머신", "캡슐/POD머신", 
"팝업토스터기", "오븐토스터기", "음식물처리기", "제빵기", "거품/반죽기", 
"전기그릴", "생선그릴", "쥬서기/녹즙기", "핸드블렌더", "슬로우쿠커", "전기냄비", 
"전기찜기", "튀김기", "전기팬", "샌드위치제조기", "와플제조기", "탄산수제조기", 
"에어프라이어", "믹서기", "전기담요", "전기방석", "전기요", "황토매트", "카페트매트", 
"옥매트", "기타전기매트", "전기장판", "전기온풍기", "가스온풍기", "석유온풍기", "돈풍기", 
"가스보일러", "석유보일러", "전기보일러", "전기온수기", "가스온수기", "라디에이터", 
"컨벡터", "스탠드형에어컨", "벽걸이형에어컨", "멀티형에어컨", "창문형에어컨", "일반용제습기", 
"냉온풍기", "자연식가습기", "가열식가습기", "초음파식가습기", "복합식가습기", "공기청정기", 
"에어워셔", "일반용냉풍기", "스탠드형선풍기", "탁상형선풍기", "휴대용선풍기", "천장형선풍기", 
"전기히터", "석유히터", "가스히터", "연탄/화목난로", "안마의자", "안마기"]
print(f"length of sample category list : {len(category_list)}")

In [ ]:
sample_cat_ids = [unq_category_ids[i] for i, x in enumerate(unq_category_levels) if x[-1] in category_list]
print(f"length of sample cat ids : {len(sample_cat_ids)}")
sample_cat_levels = [x for i, x in enumerate(unq_category_levels) if x[-1] in category_list]
print(f"length of sample cat levels : {len(sample_cat_levels)}")

In [ ]:
sample_cat_ids.extend(['50000151', '50000089', '50000088', '50000152'])
sample_cat_levels.extend(["['디지털/가전', '노트북']'", "['디지털/가전', 'PC']", "['디지털/가전', '게임기/타이틀']", "['디지털/가전', '태블릿PC']"])
print(f"length of sample cat ids : {len(sample_cat_ids)}")
print(f"length of sample cat levels : {len(sample_cat_levels)}")

In [ ]:
crawl = Crawl_Naver_Shopping(list_type="가격비교")
crawl_result = crawl.get_product_infos_in_category(cat_ids = unq_category_ids, cat_levels = unq_category_levels)

In [ ]:
pd.DataFrame(crawl_result).to_excel("네이버_쇼핑_크롤링_220426.xlsx", index =False)

In [ ]:
light_stand_df = pd.read_excel("조명_스탠드_수집_220401.xlsx")
digital_df  = pd.read_excel("가전_디지털_220401.xlsx")

In [ ]:
product_data = pd.concat([digital_df.drop_duplicates("id"), light_stand_df.drop_duplicates("id")], axis = 0)

In [ ]:
# pd_crawl_result = pd.DataFrame(crawl_result)
product_data = product_data[['searchCategoryId', 'searchCategoryName', 'id', 'productName', 'scoreInfo', 'category1Id', 'category2Id',
                             'category3Id', 'category4Id', 'category1Name', 'category2Name',
                             'category3Name', 'category4Name', 'maker', 'makerNo', 'brand',
                             'brandNo', 'price', 'priceUnit', 'lnchYm', 'productAttr', 'imageName', 'crUrl', 'imageUrl']]

In [ ]:
product_data.to_excel("네이버_쇼핑_크롤링_220404.xlsx", index = False)